### Libraries

In [37]:
import math
import numpy as np

import pandas 
import plotly.express as px

### Constants

In [38]:
# Gauss-Laguerre constants
# Notation: (abscissa, weight)
N = 6

GAUSS_LAGUERRE = [
    (0.222847, 0.458964),
    (1.188932, 0.417),
    (2.992736, 0.113373),
    (5.775144, 0.0103992),
    (9.837467, 0.000261017),
    (15.982874, 0.000000898548)
]

### Functions

In [39]:
# Weibull distribution function
def f_Weibull (x, k, mu):
    varLambdaDenom = lambda s, k : math.pow (s, 1/k)
    
    area = 0
    for i in range(N):
        area += GAUSS_LAGUERRE[i][1] * varLambdaDenom(GAUSS_LAGUERRE[i][0], k)
    
    varLambda = mu / area

    f = (k / varLambda) * math.pow(x / varLambda, k-1) * math.pow(math.e, -math.pow(x / varLambda, k))

    return f

In [40]:
# Integral of Weibull function
def cdf_Weibull (t, k, mu):
    area = 0
    
    for i in range(N):
        area += GAUSS_LAGUERRE[i][1] * f_Weibull( -GAUSS_LAGUERRE[i][0] + t, k, mu)

    return area

In [41]:
# Inverse survivability -- solved as root-finding problem using bisection method
def surv_inv (alpha, a, b, k, mu):
    isDiffSign = lambda x1, x2 : np.sign(x1) != np.sign(x2)

    f = lambda x : cdf_Weibull(x, k, mu) - alpha

    f_a = f(a)
    f_b = f(b)
    tol = 1.0e-4

    if f_a == 0.0:
        return a
    
    if f_b == 0.0:
        return b
    
    n = int (math.ceil (math.log(abs(b-a) / tol) / math.log(2)))

    for i in range(n):
        c = 0.5 * (a + b)
        f_c = f(c)

        if f_c == 0.0:
            return c

        if isDiffSign(f_a, f_c):
            a = c
            f_a = f_c
        
        elif isDiffSign(f_b, f_c):
            b = c
            f_b = f_c

    return 0.5 * (a+b)

### Evaluation

In [42]:
# Mu; average life expectancy
expected_value = 78

# List for values
y_CDF_decr = []
y_CDF_cons = []
y_CDF_incr = []
x_values = []

# For alpha in [0,1] with 0.01 step
for alpha in range(0, 100, 1):
    alpha /= 100

    area_decr = 0
    area_cons = 0
    area_incr = 0

    for i in range(N):
        area_decr += surv_inv (alpha, 0, 100, 0.5, expected_value)
        area_cons += surv_inv (alpha, 0, 100, 1, expected_value)
        area_incr += surv_inv (alpha, 0, 100, 2, expected_value)

    x_values.append(alpha)

    y_CDF_decr.append(area_decr)
    y_CDF_cons.append(area_cons)
    y_CDF_incr.append(area_incr)

ValueError: math domain error

### Plotting